In [1]:
import matplotlib
matplotlib.use('agg')

In [2]:
# imports
import pandas as pd
import numpy as np

import re
import json
import yaml

import os
import copy

from chardet.universaldetector import UniversalDetector

In [3]:
# hierarchical apply function on mixed dict/list struct
def map_nested_struct_modify(ob, func):
    if isinstance(ob,list):
        for v in ob:
            if isinstance(v,list) or isinstance(v,dict):
                map_nested_struct_modify(v, func)
            else:
                v = func(v)    
    elif isinstance(ob,dict):
        for k, v in ob.items():
            if isinstance(ob[k],list) or isinstance(ob[k],dict):
                map_nested_struct_modify(v, func)
            else:
                ob[k] = func(v)
    else:
        ob = func(ob)

In [4]:
if os.environ.get('JUPYTER_GATEWAY'):
    path='/srv/notebooks'
else:
    path='.'
    
datapath = "{}/datasets".format(path)

datasets = [
        {'id': '0', 'filename':'titanic.csv', 'desc':'Master of Life'},
        {'id': '1', 'filename':'iris.csv',    'desc':'So many flowers'},
        {'id': '2', 'filename':'pokemon.csv', 'desc':'Gotta catch \'em all!'},
        {'id': '3', 'filename':'boston.csv',  'desc':'Live and let live'}
]

df = pd.DataFrame()

df_meta = {
    'id'       : '',
    'file'     : {},
    'name'     : '',
    'desc'     : '',
    'vars'     : []
}

In [5]:
REQUEST = json.dumps({
    'path' : {},
    'args' : {}
})

In [6]:
def response_dict(d, content_type='application/json'):
    if content_type == 'application/json':
        print(json.dumps(d))
    elif content_type == 'application/xml' or content_type == 'text/xml':
        print(yaml.dump(d))

def response_meta(status=200, content_type='application/json'):
    print(json.dumps({
        "headers" : { "Content-Type" : content_type},
        "status" : status
    }))

In [7]:
def uniquify(seq): 
    checked = dict()
    uniq = []
    for e in seq:
        if e not in checked.keys():
            if e:
                checked[e]=0
                uniq.append(e)
            else:
                checked[e]=1
                uniq.append('_1')
        else:
            checked[e] += 1
            uniq.append('{}_{}'.format(e,checked[e]))
    return uniq

def to_alphanum(s):
    return re.sub(r'[^0-9A-Za-z._]+', '', s).lower()

def prep_names(seq):
    names = [ to_alphanum(x)  for x in seq]
    return uniquify(names)

def dtype_to_string(x):
    return {
        'b': 'bool',
        'i': 'long',
        'u': 'long',
        'f': 'double',
        'c': 'complex',
        'O': 'object',
        'S': 'char',
        'a': 'char',
        'U': 'string',
        'V': 'raw'
    }.get(x[1], 'unknown')

def is_number(s):
    try:
        complex(s) # for int, long, float and complex
    except ValueError:
        return False
    return True

def is_float(s):
    try:
        float(s) # for int, long, float
    except ValueError:
        return False
    return True

def is_int(s):
    try:
        return float(s).is_integer() # for int, long, float
    except ValueError:
        return False

def is_infinite(s):
    try:
        return np.isinf(float(s)) # for int, long, float
    except ValueError:
        return False

def is_zero(s):
    return not s

def vector_purity(x):
    for i in list(range(len(x))):
        # hard compressor
        x[i] = max(0, min(1, x[i]))
    return max(x)*(1 - (sum(x)-1)/(len(x)-1))

def get_typestats(sr):
    infinite= sum(sr.apply(is_infinite))
    numeric = sum(sr.apply(is_number))
    integer = sum(sr.apply(is_int))
    nan     = sum(sr.isnull())
    zero    = sum(sr.apply(is_zero))
    unique  = len(sr.unique())
    
    count   = len(sr)
    valid   = count-nan
    valid_ratio = valid/count

    numeric = numeric - integer - nan
    string = count - numeric - integer
    
    tpurity = vector_purity([integer/valid, numeric/valid, string/valid])

    # log vs linear?
    cat = False
    if (unique/valid)<0.1 :
        cat = True

    t = 'string' 
    if (integer/valid)>0.5 :
        t = 'integer'
    elif (numeric/valid)>0.5 :
        t = 'numeric'
    
    d = { 'infinite': infinite,
          'numeric' : numeric,
          'integer' : integer,
          'nan'     : nan,
          'zero'    : zero,
          'string'  : string,
          'unique'  : unique,
          'valid'   : valid,
          'quality' : valid_ratio*tpurity,
          'cat'     : cat,
          'tcoerce' : t,
          'tpurity' : tpurity,
          'type'    : dtype_to_string(sr.dtype.str)
    }
    
    return d

def numpy2py(ob):
    return np.asscalar(ob) if isinstance(ob, np.generic) else ob

def format_float(ob):
    return float(format(ob,'.2f')) if isinstance(ob, float) else ob

def numpy_tojson(ob):
    map_nested_struct_modify(ob, numpy2py)
    map_nested_struct_modify(ob, format_float)
    return ob

def sizeof_fmt(num, suffix='B'):
    for unit in ['','K','M','G','T','P','E','Z']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

def detect_encoding(filename):
    f = open(filename, 'rb')
    detector = UniversalDetector()
    for line in f:
        detector.feed(line)
        if detector.done: break
    detector.close()
    f.close()
    return detector.result

In [8]:
# TEST
#sr = pd.Series([1,0.0, '', '', np.inf, np.nan, 2.9, '0', '111', 'kkk'])
#get_typestats(sr)

In [9]:
# help functions
def rows_na_any(df):
    na_df = pd.isnull(df)
    d = na_df.apply(np.any, axis=0)
    return len(d[d==True])

def col_na_any(df):
    na_df = pd.isnull(df)
    d = na_df.apply(np.any, axis=1)
    return len(d[d==True])

In [10]:
def load_dataset(id):
    global datasets, df, df_meta

    dataset = None
    for item in datasets:
        if id == item['id']:
            dataset = item
            break;
            
    if not dataset:
        return False
    
    filename = dataset.get('filename', None)
    
    #check if it's already loaded
    if id==df_meta['id']:
        return True
    
    if filename:
        filepath = "{}/{}".format(datapath,filename)
        df = pd.read_csv(
            filepath, 
            sep=None, 
            engine='python', 
            true_values=['True', 'true'], 
            false_values=['False','false']
        )

        #dataset id
        df_meta['id']       = id
        df_meta['desc']     = item.get('desc','')
        df_meta['name']     = item.get('name',filename.split('.')[0].title())
        df_meta['vars']     = [ {'id':str(i), 'alias' : k , 'name': v} for i,k,v in zip(list(range(len(df.columns))),prep_names(df.columns),df.columns) ]
    
        df_meta['file'] = {
            'path'      : filepath,
            'name'      : item['filename'],
            'size'      : sizeof_fmt(os.path.getsize(filepath)),
            'encoding'  : detect_encoding(filepath) 
        }
        
        #rename df columns
        df.columns = prep_names(df.columns)
        
        return True
    else:
        df = pd.DataFrame()
        df_meta['id']   = id
        df_meta['file'] = {}
        df_meta['name'] = ''
        df_meta['desc'] = ''
        df_meta['vars'] = []
        
        return False

In [11]:
# TEST
#load_dataset('1')
#df_meta

In [12]:
# GET /datasets

response_dict(datasets, 'application/json')

[{"id": "0", "desc": "Master of Life", "filename": "titanic.csv"}, {"id": "1", "desc": "So many flowers", "filename": "iris.csv"}, {"id": "2", "desc": "Gotta catch 'em all!", "filename": "pokemon.csv"}, {"id": "3", "desc": "Live and let live", "filename": "boston.csv"}]


In [13]:
# ResponseInfo GET /datasets

response_meta()

{"status": 200, "headers": {"Content-Type": "application/json"}}


In [14]:
# GET /datasets/:id

if not os.environ.get('JUPYTER_GATEWAY'):
    REQUEST = json.dumps({'path': {'id':'0'}})

request = json.loads(REQUEST)
dataset_id = request['path'].get('id')

success = load_dataset(dataset_id)

if not success:
    response_dict('', 'application/json')
else:
    response_dict(df_meta, 'application/json')

{"id": "0", "vars": [{"id": "0", "name": "Unnamed: 0", "alias": "unnamed0"}, {"id": "1", "name": "Class", "alias": "class"}, {"id": "2", "name": "Sex", "alias": "sex"}, {"id": "3", "name": "Age", "alias": "age"}, {"id": "4", "name": "Survived", "alias": "survived"}, {"id": "5", "name": "Freq", "alias": "freq"}], "file": {"encoding": {"encoding": "ascii", "confidence": 1.0}, "size": "1.1KB", "path": "./datasets/titanic.csv", "name": "titanic.csv"}, "name": "Titanic", "desc": "Master of Life"}


In [15]:
# ResponseInfo GET /datasets/:id

status = 404 if not df_meta['id'] else 200
response_meta(status)

{"status": 200, "headers": {"Content-Type": "application/json"}}


In [16]:
df_meta

{'desc': 'Master of Life',
 'file': {'encoding': {'confidence': 1.0, 'encoding': 'ascii'},
  'name': 'titanic.csv',
  'path': './datasets/titanic.csv',
  'size': '1.1KB'},
 'id': '0',
 'name': 'Titanic',
 'vars': [{'alias': 'unnamed0', 'id': '0', 'name': 'Unnamed: 0'},
  {'alias': 'class', 'id': '1', 'name': 'Class'},
  {'alias': 'sex', 'id': '2', 'name': 'Sex'},
  {'alias': 'age', 'id': '3', 'name': 'Age'},
  {'alias': 'survived', 'id': '4', 'name': 'Survived'},
  {'alias': 'freq', 'id': '5', 'name': 'Freq'}]}

In [17]:
# GET /datasets/:id/stats

if not os.environ.get('JUPYTER_GATEWAY'):
    REQUEST = json.dumps({'path': {'id':'2'}})

request = json.loads(REQUEST)
dataset_id = request['path'].get('id')

success = load_dataset(dataset_id)

if not success:
    response_dict('', 'application/json')
else:

    #cell data
    shape = df.shape

    dv = []

    #extract types
    for item in df_meta['vars']:
        sr = df[item['alias']]
        dv.append(
            {
                'id'    : item['id'],
                'alias' : item['alias'],
                'name'  : item['name'],
                'type'  : get_typestats(sr),
                'sample': [str(x) for x in sr.sample(n=10).tolist()]
            }
        )
    
    ds = copy.deepcopy(df_meta)
    ds.update({
        'dims': len(shape),
        'rows': shape[0],
        'cols': shape[1],
        'na': {
            'cols': rows_na_any(df),
            'rows': col_na_any(df)
        }
    })
    
    d = {
        'ds'  : ds,
        'vars': dv
    }

    #output
    response_dict(numpy_tojson(d), 'application/json')

{"ds": {"dims": 2, "rows": 800, "file": {"encoding": {"encoding": "utf-8", "confidence": 0.94}, "name": "pokemon.csv", "path": "./datasets/pokemon.csv", "size": "43.0KB"}, "name": "Pokemon", "desc": "Gotta catch 'em all!", "vars": [{"id": "0", "alias": "_1", "name": "#"}, {"id": "1", "alias": "name", "name": "Name"}, {"id": "2", "alias": "type1", "name": "Type 1"}, {"id": "3", "alias": "type2", "name": "Type 2"}, {"id": "4", "alias": "total", "name": "Total"}, {"id": "5", "alias": "hp", "name": "HP"}, {"id": "6", "alias": "attack", "name": "Attack"}, {"id": "7", "alias": "defense", "name": "Defense"}, {"id": "8", "alias": "sp.atk", "name": "Sp. Atk"}, {"id": "9", "alias": "sp.def", "name": "Sp. Def"}, {"id": "10", "alias": "speed", "name": "Speed"}, {"id": "11", "alias": "generation", "name": "Generation"}, {"id": "12", "alias": "legendary", "name": "Legendary"}], "id": "2", "na": {"rows": 386, "cols": 1}, "cols": 13}, "vars": [{"id": "0", "name": "#", "alias": "_1", "type": {"string":

In [18]:
# ResponseInfo GET /datasets/:id/stats

status = 404 if not df_meta['id'] else 200
response_meta()

{"status": 200, "headers": {"Content-Type": "application/json"}}


In [19]:
def describe(sr):
    d = dict(sr.describe())
    if '25%' in d.keys() : d['q25'] = d.pop('25%')
    if '50%' in d.keys() : d['q50'] = d.pop('50%')
    if '75%' in d.keys() : d['q75'] = d.pop('75%')
    
    if '50%' in d.keys() :
        d['skew']  = sr.skew()
        d['kurt]'] = sr.kurt()
        d['mean]'] = sr.mean()
    
    return d

def get_stats(sr, t, cat):
    if cat:
        sr = sr[sr.notnull()].apply(lambda x: str(x))
        d = describe(sr)
        d.update({
            'categories': list(sr.unique()),
            'histogram': [{'v': str(x), 'c': y } for x,y in sr.value_counts().items()]
        })
    elif t=='integer' or t=='numeric':
        d = describe(sr)
        
        iqr = d['q75']-d['q25']
        w_hi = sr[(sr>d['q75']) & (sr<d['q50']+1.5*iqr)].max()
        w_lo = sr[(sr<d['q25']) & (sr>d['q50']-1.5*iqr)].min()

        a_hi = sr[sr>w_hi].count()
        a_lo = sr[sr<w_lo].count()

        d.update({
            'iqr'      : iqr,
            'whiskers' : { 'hi':w_hi, 'lo':w_lo },
            'anomalies': { 'hi':a_hi, 'lo':a_lo }
        })
        
        d.update({
            'histogram': [{'v': x, 'c': y } for x,y in sr.value_counts(bins=20, sort=False).items()]
        })
    else:
        sr = sr.apply(lambda x: str(x))
        d = describe(sr)
    return d

In [20]:
# GET /datasets/:id/variables/:vid

if not os.environ.get('JUPYTER_GATEWAY'):
    REQUEST = json.dumps({'path': {'id':'2', 'vid':'3'}})

request = json.loads(REQUEST)
id = request['path'].get('id')
vid = request['path'].get('vid')

success = load_dataset(id)

if not success:
    response_dict('', 'application/json')
else:

    dv = {
        'id':vid
    }

    #extract types
    for item in df_meta['vars']:
        if vid == item['id']:
            sr = df[item['alias']]
            vardet = get_typestats(sr)
            dv.update(
                {
                    'id'    : item['id'],
                    'alias' : item['alias'],
                    'name'  : item['name'],
                    'type'  : vardet,
                    'sample': [str(x) for x in sr.sample(n=25).tolist()],
                    'stats' : get_stats(sr, vardet['tcoerce'], vardet['cat'])
                }
            )
            break
    
    id_num = int(float(vid))
    nav_prev = id_num-1 if 0 <= (id_num-1) else None
    nav_next = id_num+1 if (id_num+1)< len(df_meta['vars']) else None
    
    
    ds = copy.deepcopy(df_meta)
    
    d = {
        'ds'  : ds,
        'var': dv,
        'nav': {
            'prev':nav_prev,
            'next':nav_next
        }
    }

    #output
    response_dict(numpy_tojson(d), 'application/json')

{"ds": {"id": "2", "vars": [{"id": "0", "alias": "_1", "name": "#"}, {"id": "1", "alias": "name", "name": "Name"}, {"id": "2", "alias": "type1", "name": "Type 1"}, {"id": "3", "alias": "type2", "name": "Type 2"}, {"id": "4", "alias": "total", "name": "Total"}, {"id": "5", "alias": "hp", "name": "HP"}, {"id": "6", "alias": "attack", "name": "Attack"}, {"id": "7", "alias": "defense", "name": "Defense"}, {"id": "8", "alias": "sp.atk", "name": "Sp. Atk"}, {"id": "9", "alias": "sp.def", "name": "Sp. Def"}, {"id": "10", "alias": "speed", "name": "Speed"}, {"id": "11", "alias": "generation", "name": "Generation"}, {"id": "12", "alias": "legendary", "name": "Legendary"}], "file": {"encoding": {"encoding": "utf-8", "confidence": 0.94}, "name": "pokemon.csv", "path": "./datasets/pokemon.csv", "size": "43.0KB"}, "name": "Pokemon", "desc": "Gotta catch 'em all!"}, "var": {"stats": {"categories": ["Poison", "Flying", "Dragon", "Ground", "Fairy", "Grass", "Fighting", "Psychic", "Steel", "Ice", "Rock

In [21]:
# ResponseInfo GET /datasets/:id/variables/:vid

response_meta()

{"status": 200, "headers": {"Content-Type": "application/json"}}


In [23]:
# GET /test
import seaborn as sns
import sys

d = get_stats(df['type2'], 'string', True)['histogram']
dd = pd.DataFrame(d)
sns.set()
s = sns.barplot(x='v', y='c', data=dd)
#s.figure.savefig(sys.stdout)

In [ ]:
# ResponseInfo GET /test

response_meta(200, 'image/png')